# 프롬프트 선정하기 (영어 -> 한국어) 


In [6]:
!pip install --upgrade openai
!pip install dotenv

  Obtaining dependency information for dotenv from https://files.pythonhosted.org/packages/b2/b7/545d2c10c1fc15e48653c91efde329a790f2eecfbbf2bd16003b5db2bab0/dotenv-0.9.9-py2.py3-none-any.whl.metadata
  Obtaining dependency information for python-dotenv from https://files.pythonhosted.org/packages/5f/ed/539768cf28c661b5b068d66d96a2f155c4971a5d55684a514c1a0e0dec2f/python_dotenv-1.1.1-py3-none-any.whl.metadata


In [7]:
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

# API 키 로드
load_dotenv()
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")

# Solar API 클라이언트 초기화
client = OpenAI(
    api_key=UPSTAGE_API_KEY,
    base_url="https://api.upstage.ai/v1/solar"
)

print("✅ 설정이 완료되었습니다.")

✅ 설정이 완료되었습니다.


In [18]:
# 테스트할 영어 요약문 샘플
sample_data = {
    'english_summary': [
        "Rose invited Jack to join her for dinner, and Jack agreed to the invitation.",
        "#Person2# advises #Person1# to wear a suit and tie to their job interview.",
        "#Person1# conducted an interview with John regarding his motivation, his management style, and his plans for the future."
    ]
}
test_df = pd.DataFrame(sample_data)

print("✅ 테스트할 샘플 데이터 준비 완료:")
test_df

✅ 테스트할 샘플 데이터 준비 완료:


,english_summary
0,"Rose invited Jack to join her for dinner, and ..."
1,#Person2# advises #Person1# to wear a suit and...
2,#Person1# conducted an interview with John reg...


In [19]:
# 네가 제안한 3가지 프롬프트 아이디어
PROMPT_TEMPLATE_1 = """
Translate the following English dialogue summary into Korean.

Your translation must be:
- **Detailed and descriptive** (avoid shortening the content).
- **Fluent and natural** in Korean.
- **Complete sentences** should be used, not fragmented or overly simplified.
- Keep speaker tags like #Person1#, #person2# exactly as they are.
- Translate personal names phonetically (e.g., "Francis" → "프랜시스").

Focus on maintaining all intentions, events, and emotional cues from the original. DO NOT summarize.

English Text to Translate:
{text_to_translate}
"""

PROMPT_TEMPLATE_2 = """
You are a Korean language expert who specializes in summarizing everyday conversations naturally and clearly.

Translate the following English summary into Korean as if you are summarizing a casual spoken conversation between two people. The tone should be natural but still clear and informative.

**Guidelines:**
1. Keep the speaker markers (e.g., #Person1#, #person2#) exactly as they are — do not translate or alter them.
2. Translate names phonetically. Do NOT replace them with other figures or titles (e.g., "Francis" → "프랜시스").
3. Use natural Korean phrasing, just like how people explain a conversation to a friend — but avoid slang or overly casual words.
4. Don’t skip important details — the summary should be concise but complete enough to understand the main points.

---

English Text to Translate:
{text_to_translate}
"""

PROMPT_TEMPLATE_3 = """
You are a professional translator working on a Korean language dataset for AI training.

Translate the following English summary into **natural, fluent, and detailed Korean**.

**Instructions:**
1. Keep speaker tags such as `#Person1#`, `#person2#` **exactly as they are**.
2. Personal names must be translated **phonetically** (e.g., "Francis" → "프랜시스").
3. Use a **formal**, full-sentence tone.
4. DO NOT summarize or skip information. Be as detailed as the source.

**Example Input:**
"#Person1# explained to #person2# why they were late for the meeting, and #person2# acknowledged the situation with understanding."

**Example Output:**
"#Person1#은 회의에 지각한 이유를 #person2#에게 자세히 설명했고, #person2#는 그 상황을 이해하는 반응을 보였다."

---

Please translate the following English summary:
{text_to_translate}
"""

# 프롬프트들을 딕셔너리로 묶어서 관리
prompt_templates = {
    "Prompt 1 (상세 묘사)": PROMPT_TEMPLATE_1,
    "Prompt 2 (자연스러운 톤)": PROMPT_TEMPLATE_2,
    "Prompt 3 (전문 번역가 톤)": PROMPT_TEMPLATE_3,
}

print("✅ 3가지 번역 프롬프트 템플릿 준비 완료.")

✅ 3가지 번역 프롬프트 템플릿 준비 완료.


In [20]:
# 네가 제안한 3가지 프롬프트 아이디어
PROMPT_TEMPLATE_1 = """
Translate the following English dialogue summary into Korean.

Your translation must be:
- **Detailed and descriptive** (avoid shortening the content).
- **Fluent and natural** in Korean.
- **Complete sentences** should be used, not fragmented or overly simplified.
- Keep speaker tags like #Person1#, #person2# exactly as they are.
- Translate personal names phonetically (e.g., "Francis" → "프랜시스").

Focus on maintaining all intentions, events, and emotional cues from the original. DO NOT summarize.

English Text to Translate:
{text_to_translate}
"""

PROMPT_TEMPLATE_2 = """
You are a Korean language expert who specializes in summarizing everyday conversations naturally and clearly.

Translate the following English summary into Korean as if you are summarizing a casual spoken conversation between two people. The tone should be natural but still clear and informative.

**Guidelines:**
1. Keep the speaker markers (e.g., #Person1#, #person2#) exactly as they are — do not translate or alter them.
2. Translate names phonetically. Do NOT replace them with other figures or titles (e.g., "Francis" → "프랜시스").
3. Use natural Korean phrasing, just like how people explain a conversation to a friend — but avoid slang or overly casual words.
4. Don’t skip important details — the summary should be concise but complete enough to understand the main points.

---

English Text to Translate:
{text_to_translate}
"""

PROMPT_TEMPLATE_3 = """
You are a professional translator working on a Korean language dataset for AI training.

Translate the following English summary into **natural, fluent, and detailed Korean**.

**Instructions:**
1. Keep speaker tags such as `#Person1#`, `#person2#` **exactly as they are**.
2. Personal names must be translated **phonetically** (e.g., "Francis" → "프랜시스").
3. Use a **formal**, full-sentence tone.
4. DO NOT summarize or skip information. Be as detailed as the source.

**Example Input:**
"#Person1# explained to #person2# why they were late for the meeting, and #person2# acknowledged the situation with understanding."

**Example Output:**
"#Person1#은 회의에 지각한 이유를 #person2#에게 자세히 설명했고, #person2#는 그 상황을 이해하는 반응을 보였다."

---

Please translate the following English summary:
{text_to_translate}
"""

# 프롬프트들을 딕셔너리로 묶어서 관리
prompt_templates = {
    "Prompt 1 (상세 묘사)": PROMPT_TEMPLATE_1,
    "Prompt 2 (자연스러운 톤)": PROMPT_TEMPLATE_2,
    "Prompt 3 (전문 번역가 톤)": PROMPT_TEMPLATE_3,
}

print("✅ 3가지 번역 프롬프트 템플릿 준비 완료.")

✅ 3가지 번역 프롬프트 템플릿 준비 완료.


In [21]:
from tqdm.notebook import tqdm
import pandas as pd

results = []
# 각 샘플 문장에 대해
for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="샘플 번역 중"):
    english_summary = row['english_summary']
    result_row = {'Original English': english_summary}
    
    # 3가지 프롬프트를 모두 테스트
    for name, template in prompt_templates.items():
        korean_translation = get_translation(english_summary, template)
        result_row[name] = korean_translation
    
    results.append(result_row)

print("\n✅ 모든 프롬프트에 대한 번역 테스트 완료!")

# 결과를 DataFrame으로 변환하여 보기 좋게 출력
results_df = pd.DataFrame(results)

# 셀 내용이 잘리지 않게 출력 옵션 설정
pd.set_option('display.max_colwidth', None)

print("\n--- 번역 프롬프트 비교 결과 ---")
results_df

샘플 번역 중:   0%|          | 0/3 [00:00<?, ?it/s]

NameError: name 'get_translation' is not defined

In [9]:
# 네가 제안한 3가지 프롬프트 아이디어
PROMPT_TEMPLATE_1 = """
Translate the following English dialogue summary into Korean.

Your translation must be:
- **Detailed and descriptive** (avoid shortening the content).
- **Fluent and natural** in Korean.
- **Complete sentences** should be used, not fragmented or overly simplified.
- Keep speaker tags like #Person1#, #person2# exactly as they are.
- Translate personal names phonetically (e.g., "Francis" → "프랜시스").

Focus on maintaining all intentions, events, and emotional cues from the original. DO NOT summarize.

English Text to Translate:
{text_to_translate}
"""

PROMPT_TEMPLATE_2 = """
You are a Korean language expert who specializes in summarizing everyday conversations naturally and clearly.

Translate the following English summary into Korean as if you are summarizing a casual spoken conversation between two people. The tone should be natural but still clear and informative.

**Guidelines:**
1. Keep the speaker markers (e.g., #Person1#, #person2#) exactly as they are — do not translate or alter them.
2. Translate names phonetically. Do NOT replace them with other figures or titles (e.g., "Francis" → "프랜시스").
3. Use natural Korean phrasing, just like how people explain a conversation to a friend — but avoid slang or overly casual words.
4. Don’t skip important details — the summary should be concise but complete enough to understand the main points.

---

English Text to Translate:
{text_to_translate}
"""

PROMPT_TEMPLATE_3 = """
You are a professional translator working on a Korean language dataset for AI training.

Translate the following English summary into **natural, fluent, and detailed Korean**.

**Instructions:**
1. Keep speaker tags such as `#Person1#`, `#person2#` **exactly as they are**.
2. Personal names must be translated **phonetically** (e.g., "Francis" → "프랜시스").
3. Use a **formal**, full-sentence tone.
4. DO NOT summarize or skip information. Be as detailed as the source.

**Example Input:**
"#Person1# explained to #person2# why they were late for the meeting, and #person2# acknowledged the situation with understanding."

**Example Output:**
"#Person1#은 회의에 지각한 이유를 #person2#에게 자세히 설명했고, #person2#는 그 상황을 이해하는 반응을 보였다."

---

Please translate the following English summary:
{text_to_translate}
"""

# 프롬프트들을 딕셔너리로 묶어서 관리
prompt_templates = {
    "Prompt 1 (상세 묘사)": PROMPT_TEMPLATE_1,
    "Prompt 2 (자연스러운 톤)": PROMPT_TEMPLATE_2,
    "Prompt 3 (전문 번역가 톤)": PROMPT_TEMPLATE_3,
}

print("✅ 3가지 번역 프롬프트 템플릿 준비 완료.")

✅ 3가지 번역 프롬프트 템플릿 준비 완료.


In [22]:
# 4번 셀: API 호출 함수 정의
def get_translation(text_to_translate, prompt_template):
    """주어진 텍스트와 프롬프트 템플릿으로 Solar API 번역을 요청하는 함수"""
    # placeholder 이름이 다를 수 있으므로, 유연하게 처리
    if '{text_to_translate}' in prompt_template:
        prompt = prompt_template.format(text_to_translate=text_to_translate)
    else:
        prompt = prompt_template.format(dialogue_to_summarize=text_to_translate)

    try:
        response = client.chat.completions.create(
            model="solar-1-mini-chat",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1 # 일관된 비교를 위해 temperature를 낮게 설정
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"API Error: {e}"

print("✅ API 호출 함수 준비 완료.")

✅ API 호출 함수 준비 완료.


In [23]:
from tqdm.notebook import tqdm
import pandas as pd

results = []
# 각 샘플 문장에 대해
for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="샘플 번역 중"):
    english_summary = row['english_summary']
    result_row = {'Original English': english_summary}
    
    # 3가지 프롬프트를 모두 테스트
    for name, template in prompt_templates.items():
        korean_translation = get_translation(english_summary, template)
        result_row[name] = korean_translation
    
    results.append(result_row)

print("\n✅ 모든 프롬프트에 대한 번역 테스트 완료!")

# 결과를 DataFrame으로 변환하여 보기 좋게 출력
results_df = pd.DataFrame(results)

# 셀 내용이 잘리지 않게 출력 옵션 설정
pd.set_option('display.max_colwidth', None)

print("\n--- 번역 프롬프트 비교 결과 ---")
results_df

샘플 번역 중:   0%|          | 0/3 [00:00<?, ?it/s]


✅ 모든 프롬프트에 대한 번역 테스트 완료!

--- 번역 프롬프트 비교 결과 ---


,Original English,Prompt 1 (상세 묘사),Prompt 2 (자연스러운 톤),Prompt 3 (전문 번역가 톤)
0,"Rose invited Jack to join her for dinner, and Jack agreed to the invitation.","여러분이 요청하신 대로 아래와 같이 번역하겠습니다.\n\n여러분의 번역은 다음과 같아야 합니다:\n- **세부적이고 설명적인** (내용을 축약하지 않은) 번역.\n- 한국어로 **유창하고 자연스러운** 문장.\n- 단편적이거나 과도하게 단순화되지 않은 **완전한 문장**.\n- #사람1#, #사람2#와 같은 화자 표시는 그대로 유지.\n- 인명 등은 음역하여 표기 (예: ""Francis"" → ""프랜시스"").\n\n원문의 모든 의도, 사건, 감정적 단서를 유지하는 것에 집중해 주세요. 요약하지 마세요.\n\n번역할 영어 원문:\n로즈는 잭을 저녁 식사에 초대했고, 잭은 그 초대에 동의했습니다.\n\n한국어 번역:\n로즈는 잭을 함께 저녁 식사에 초대했고, 잭은 그 초대에 응하기로 했습니다.","#Person1#이 로즈에게 같이 저녁을 먹자고 초대했고, #person2#인 잭은 그 초대에 응했어요.","로즈는 잭에게 저녁 식사에 함께 할 것을 제안하였고, 잭은 그 초대에 응했습니다."
1,#Person2# advises #Person1# to wear a suit and tie to their job interview.,#Person2#는 #Person1#에게 그 취업 면접에 정장과 넥타이를 착용하라고 조언한다.,#Person2#는 #Person1#에게 면접 때 정장에 넥타이를 매고 가라고 조언했어.,"""#Person2#는 #Person1#에게 그들의 취업 면접에 정장과 넥타이를 착용할 것을 권유합니다."""
2,"#Person1# conducted an interview with John regarding his motivation, his management style, and his plans for the future.","#Person1#은 존과의 인터뷰에서 그의 동기부여, 그의 경영 스타일, 그리고 앞으로의 계획에 관해 진행하였다.\n\n번역 시 다음 사항을 꼭 지켜주세요:\n- **세밀하고 자세하게** (내용을 줄이지 않고) 번역해 주세요.\n- **자연스러운 한국어로** 번역해 주세요. (의역도 가능)\n- **완전한 문장**으로 번역해 주세요, 단편적이거나 과도하게 간결하게 번역하지 말아 주세요.\n- 화자 표시인 #Person1#, #person2# 등은 그대로 유지해 주세요.\n- 인명 등은 한국어로 발음나는 대로 표기해 주세요 (예: ""Francis"" → ""프랜시스"")\n\n원문의 모든 의도, 사건, 감정적인 뉘앙스 등을 유지해 주세요. 요약하지 말아 주세요.\n\n번역할 원문:\n#Person1#은 존과의 인터뷰에서 그의 동기부여, 그의 경영 스타일, 그리고 앞으로의 계획에 관해 진행하였다.","#Person1#은 존과 그의 동기부여, 경영 스타일, 그리고 앞으로의 계획에 관한 인터뷰를 진행했습니다.","""#Person1#은 John과 그의 동기에 관한 인터뷰를 진행하였으며, 그의 경영 방식과 앞으로의 계획에 대해서도 이야기를 나누었습니다."""


In [24]:
PROMPT_IDEA_4 = """
# Instruction
Translate the following English text into professional, natural-sounding Korean.

# Constraints
- Keep speaker tags like #Person1# exactly as they are.
- Translate names phonetically.
- Do not add any extra information or summaries.

# English Text
{text_to_translate}

# Korean Translation
"""
PROMPT_IDEA_5 = """
You are a meticulous translator. Your task is to translate an English summary into Korean.

First, identify the key entities (people, places) and the main action in the English text.
Second, construct a natural and accurate Korean sentence that captures these elements and the original nuance.
Keep all speaker tags like #Person1# unchanged.

English: "{text_to_translate}"
Korean:
"""
PROMPT_IDEA_6 = """
You are a professional post-editor for an AI translation company. The following English text has been pre-translated into a literal Korean draft. Your job is to refine the draft into a perfectly fluent and natural final version, while keeping all original information and speaker tags like #Person1#.

English Source: "{text_to_translate}"

Refined Korean Translation:
"""

In [25]:
# 3번 셀 (2차 실험용)

# --- 1차 실험의 챔피언과 새로운 도전자 3명 ---

# 1차 챔피언 (기준점)
PROMPT_IDEA_3 = """
You are a professional translator working on a Korean language dataset for AI training.

Translate the following English summary into **natural, fluent, and detailed Korean**.

**Instructions:**
1. Keep speaker tags such as `#Person1#`, `#person2#` **exactly as they are**.
2. Personal names must be translated **phonetically** (e.g., "Francis" → "프랜시스").
3. Use a **formal**, full-sentence tone.
4. DO NOT summarize or skip information. Be as detailed as the source.

**Example Input:**
"#Person1# explained to #person2# why they were late for the meeting, and #person2# acknowledged the situation with understanding."

**Example Output:**
"#Person1#은 회의에 지각한 이유를 #person2#에게 자세히 설명했고, #person2#는 그 상황을 이해하는 반응을 보였다."

"""

# 새로운 도전자 1
PROMPT_IDEA_4 = """
# Instruction
Translate the following English text into professional, natural-sounding Korean.
# Constraints
- Keep speaker tags like #Person1# exactly as they are.
- Translate names phonetically.
- Do not add any extra information or summaries.
# English Text
{text_to_translate}
# Korean Translation
"""

# 새로운 도전자 2
PROMPT_IDEA_5 = """
You are a meticulous translator. Your task is to translate an English summary into Korean.

First, identify the key entities (people, places) and the main action in the English text.
Second, construct a natural and accurate Korean sentence that captures these elements and the original nuance.
Keep all speaker tags like #Person1# unchanged.

English: "{text_to_translate}"
Korean:
"""

# 새로운 도전자 3
PROMPT_IDEA_6 = """
You are a professional post-editor for an AI translation company. The following English text has been pre-translated into a literal Korean draft. Your job is to refine the draft into a perfectly fluent and natural final version, while keeping all original information and speaker tags like #Person1#.

English Source: "{text_to_translate}"

Refined Korean Translation:
"""
# 프롬프트들을 딕셔너리로 묶어서 관리
prompt_templates = {
    "Prompt 3 (1차 챔피언)": PROMPT_TEMPLATE_3,
    "Prompt 4 (미니멀리스트)": PROMPT_IDEA_4,
    "Prompt 5 (사고 과정 유도)": PROMPT_IDEA_5,
    "Prompt 6 (후편집 전문가)": PROMPT_IDEA_6,
}

print("✅ 2차 실험을 위한 4가지 프롬프트 템플릿 준비 완료.")

✅ 2차 실험을 위한 4가지 프롬프트 템플릿 준비 완료.


In [27]:
from tqdm.notebook import tqdm
import pandas as pd

results = []
# 각 샘플 문장에 대해
for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="샘플 번역 중"):
    english_summary = row['english_summary']
    result_row = {'Original English': english_summary}
    
    # 3가지 프롬프트를 모두 테스트
    for name, template in prompt_templates.items():
        korean_translation = get_translation(english_summary, template)
        result_row[name] = korean_translation
    
    results.append(result_row)

print("\n✅ 모든 프롬프트에 대한 번역 테스트 완료!")

# 결과를 DataFrame으로 변환하여 보기 좋게 출력
results_df = pd.DataFrame(results)

# 셀 내용이 잘리지 않게 출력 옵션 설정
pd.set_option('display.max_colwidth', None)

print("\n--- 번역 프롬프트 비교 결과 ---")
results_df

샘플 번역 중:   0%|          | 0/3 [00:00<?, ?it/s]


✅ 모든 프롬프트에 대한 번역 테스트 완료!

--- 번역 프롬프트 비교 결과 ---


,Original English,Prompt 3 (1차 챔피언),Prompt 4 (미니멀리스트),Prompt 5 (사고 과정 유도),Prompt 6 (후편집 전문가)
0,"Rose invited Jack to join her for dinner, and Jack agreed to the invitation.","로즈는 잭에게 저녁 식사에 함께 할 것을 제안했고, 잭은 그 초대에 응했습니다.","로즈는 잭에게 저녁 식사에 함께 할 것을 제안했고, 잭은 그 제안을 받아들였습니다.\n# 한국어 번역\n","#Person1#인 Rose가 #Person2#인 Jack에게 저녁 식사에 함께 할 것을 제안했고, #Person2#인 Jack은 그 제안에 동의했습니다.","#Person1#은 잭을 저녁 식사에 초대했고, 잭은 그 초대에 응했어요."
1,#Person2# advises #Person1# to wear a suit and tie to their job interview.,"""#Person2#는 #Person1#에게 그들의 취업 면접에 정장과 넥타이를 착용할 것을 권유합니다.""",# 지시사항\n다음 영어 문장을 전문적이고 자연스러운 한국어로 번역해 주세요.\n# 제약사항\n- #사람2# 와 같은 화자 표시는 그대로 유지.\n- 이름은 음역으로 표기.\n- 추가적인 정보나 요약은 하지 않음.\n# 영어 원문\n#Person2# advises #Person1# to wear a suit and tie to their job interview.\n# 한국어 번역\n,"""#Person2#는 #Person1#에게 면접에는 정장과 넥타이가 좋다고 조언합니다.""",#Person2#는 #Person1#에게 면접 때 정장과 넥타이를 착용하라고 조언한다.
2,"#Person1# conducted an interview with John regarding his motivation, his management style, and his plans for the future.","""#Person1#은 John과 그의 동기에 관한 인터뷰를 진행했으며, 그의 경영 방식과 앞으로의 계획에 대해서도 질문하였습니다.""","# 지시사항\n다음 영어 문장을 전문적이고 자연스러운 한국어로 번역해 주세요.\n# 제약사항\n- #사람1# 같은 화자 표시는 그대로 유지.\n- 이름은 음역으로 표기.\n- 추가적인 정보나 요약은 하지 않음.\n# 영어 원문\n#Person1# conducted an interview with John regarding his motivation, his management style, and his plans for the future.\n# 한국어 번역\n","#Person1#은 존과 그의 동기부여, 경영 스타일, 그리고 앞으로의 계획에 관한 인터뷰를 진행했습니다.","#Person1#은 존과 그의 동기, 경영 스타일, 그리고 앞으로의 계획에 관해 인터뷰를 진행했습니다."


In [28]:
# --- 최종 대결에 사용할 챔피언 프롬프트들과 퓨샷 예제 ---

# 실험 A를 위한 '최고의 번역 프롬프트' (2차 실험의 승자)
CHAMPION_TRANSLATION_PROMPT = """
You are a professional translator working on a Korean language dataset for AI training.
Translate the following English summary into **natural, fluent, and detailed Korean**.
**Instructions:**
1. Keep speaker tags such as `#Person1#`, `#person2#` **exactly as they are**.
2. Personal names must be translated **phonetically** (e.g., "Francis" → "프랜시스").
3. Use a **formal**, full-sentence tone.
4. DO NOT summarize or skip information. Be as detailed as the source.
---
Please translate the following English summary:
{text_to_translate}
"""

# 실험 B를 위한 '퓨샷 요약 프롬프트'
FEW_SHOT_SUMMARY_PROMPT = f"""
You are a expert in the field of dialogue summarization. Following the sample's style, provide a concise Korean summary of the given dialogue.

Sample Dialogue:
{sample_dialogue_for_fewshot}

Sample Summary (Korean):
{get_translation(sample_summary_for_fewshot, CHAMPION_TRANSLATION_PROMPT)}

---

Dialogue to summarize:
{{dialogue_to_summarize}}

Korean Summary:
"""

print("✅ 최종 대결을 위한 챔피언 프롬프트 2개 준비 완료.")

✅ 최종 대결을 위한 챔피언 프롬프트 2개 준비 완료.


In [29]:
from tqdm.notebook import tqdm
import pandas as pd
from IPython.display import display

final_comparison_results = []
# 각 샘플 데이터에 대해
for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="최종 대결 진행 중"):
    english_dialogue = row['english_dialogue']
    our_model_english_summary = row['english_summary_from_our_model']
    
    # --- 방법 A: 우리 모델(영어 요약) + 챔피언 번역 프롬프트 ---
    method_A_summary = get_translation(our_model_english_summary, CHAMPION_TRANSLATION_PROMPT)
    
    # --- 방법 B: Solar API + 퓨샷 요약 프롬프트 ---
    method_B_summary = get_translation(english_dialogue, FEW_SHOT_SUMMARY_PROMPT)

    result_row = {
        'Original English Dialogue': english_dialogue,
        'Method A (Finetuned Model + Translate)': method_A_summary,
        'Method B (Solar API Direct Summary)': method_B_summary
    }
    final_comparison_results.append(result_row)

print("\n✅ 최종 대결 완료!")

# 결과 출력
final_results_df = pd.DataFrame(final_comparison_results)
pd.set_option('display.max_colwidth', None)

print("\n--- 최종 대결: 우리 모델 vs Solar API ---")
display(final_results_df)

최종 대결 진행 중:   0%|          | 0/3 [00:00<?, ?it/s]

KeyError: 'english_dialogue'

In [30]:
import pandas as pd
from IPython.display import display
from tqdm.notebook import tqdm

# --- 1. 데이터 준비 (셀 안에서 새로 정의해서 에러 방지) ---
print("✅ 테스트용 샘플 데이터를 다시 준비합니다...")
sample_data = {
    'english_dialogue': [
        "summarize: topic: Evening Invitation dialogue: #Person1#: Hello, is Jack there? #Person2#: This is him. #Person1#: Jack! It's Rose. #Person2#: Hey, Rose. How have you been? #Person1#: I've been great, thanks for asking. I'm hosting a few friends for dinner this Saturday. I was wondering if you could join us. #Person2#: That sounds great. What time should I arrive? #Person1#: Six o'clock works for me.",
        "summarize: topic: Job Interview Preparation dialogue: #Person1#: Hey, what should I wear to a job interview? #Person2#: I'd recommend going with a suit and tie. #Person1#: I'm worried I might get too nervous during the interview. #Person2#: Don't worry about it. Just do your best and showcase yourself confidently.",
    ],
    'english_summary_from_our_model': [
        "Rose invited Jack to join her for dinner, and Jack agreed to the invitation.",
        "#Person2# advises #Person1# to wear a suit and tie to their job interview.",
    ]
}
test_df = pd.DataFrame(sample_data)
print("✅ 샘플 데이터 준비 완료.")

# --- 2. 챔피언 프롬프트들과 퓨샷 예제 정의 ---
# 퓨샷에 사용할 예제
sample_dialogue_for_fewshot = test_df.iloc[0]['english_dialogue']
sample_summary_for_fewshot_en = test_df.iloc[0]['english_summary_from_our_model']
# 예제 요약문도 한국어로 미리 번역해두자.
sample_summary_for_fewshot_ko = "로즈는 잭을 저녁 식사에 초대했고, 잭은 그 초대에 동의했습니다."


# 실험 A를 위한 '최고의 번역 프롬프트'
CHAMPION_TRANSLATION_PROMPT = """
You are a professional translator working on a Korean language dataset for AI training.
Translate the following English summary into **natural, fluent, and detailed Korean**.
**Instructions:**
1. Keep speaker tags such as `#Person1#`, `#person2#` **exactly as they are**.
2. Personal names must be translated **phonetically** (e.g., "Francis" → "프랜시스").
3. Use a **formal**, full-sentence tone.
4. DO NOT summarize or skip information. Be as detailed as the source.
---
Please translate the following English summary:
{text_to_translate}
"""

# 실험 B를 위한 '퓨샷 요약 프롬프트'
FEW_SHOT_SUMMARY_PROMPT = f"""
You are a expert in the field of dialogue summarization. Following the sample's style, provide a concise Korean summary of the given dialogue.

Sample Dialogue:
{sample_dialogue_for_fewshot}

Sample Summary (Korean):
{sample_summary_for_fewshot_ko}

---
Dialogue to summarize:
{{dialogue_to_summarize}}

Korean Summary:
"""

print("✅ 최종 대결을 위한 챔피언 프롬프트 2개 준비 완료.")


# --- 3. 실험 실행 ---
print("\n🚀 최종 대결을 시작합니다...")
final_comparison_results = []
# 각 샘플 데이터에 대해
for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="최종 대결 진행 중"):
    english_dialogue = row['english_dialogue']
    our_model_english_summary = row['english_summary_from_our_model']
    
    # --- 방법 A: 우리 모델(영어 요약) + 챔피언 번역 프롬프트 ---
    method_A_summary = get_translation(our_model_english_summary, CHAMPION_TRANSLATION_PROMPT)
    
    # --- 방법 B: Solar API + 퓨샷 요약 프롬프트 ---
    method_B_summary = get_translation(english_dialogue, FEW_SHOT_SUMMARY_PROMPT)

    result_row = {
        'Original English Dialogue': english_dialogue,
        'Method A (Finetuned + Translate)': method_A_summary,
        'Method B (Solar API Direct Summary)': method_B_summary
    }
    final_comparison_results.append(result_row)

print("\n✅ 최종 대결 완료!")

# --- 4. 결과 출력 ---
final_results_df = pd.DataFrame(final_comparison_results)
pd.set_option('display.max_colwidth', None)

print("\n--- 최종 대결: 우리 모델 vs Solar API ---")
display(final_results_df)

✅ 테스트용 샘플 데이터를 다시 준비합니다...
✅ 샘플 데이터 준비 완료.
✅ 최종 대결을 위한 챔피언 프롬프트 2개 준비 완료.

🚀 최종 대결을 시작합니다...


최종 대결 진행 중:   0%|          | 0/2 [00:00<?, ?it/s]


✅ 최종 대결 완료!

--- 최종 대결: 우리 모델 vs Solar API ---


,Original English Dialogue,Method A (Finetuned + Translate),Method B (Solar API Direct Summary)
0,"summarize: topic: Evening Invitation dialogue: #Person1#: Hello, is Jack there? #Person2#: This is him. #Person1#: Jack! It's Rose. #Person2#: Hey, Rose. How have you been? #Person1#: I've been great, thanks for asking. I'm hosting a few friends for dinner this Saturday. I was wondering if you could join us. #Person2#: That sounds great. What time should I arrive? #Person1#: Six o'clock works for me.","로즈는 잭을 저녁 식사에 초대했고, 잭은 그 초대에 응했습니다.\n","로즈는 잭을 토요일 저녁 식사에 초대했고, 잭은 초대에 동의했습니다. 만남 시간은 6시로 정해졌습니다."
1,"summarize: topic: Job Interview Preparation dialogue: #Person1#: Hey, what should I wear to a job interview? #Person2#: I'd recommend going with a suit and tie. #Person1#: I'm worried I might get too nervous during the interview. #Person2#: Don't worry about it. Just do your best and showcase yourself confidently.",#Person2#는 #Person1#에게 그들의 취업 면접에 정장과 넥타이를 착용하라고 조언합니다.,"대화1은 면접 복장에 대한 조언을 구하는 내용이고, 대화2는 면접 중 긴장하는 것에 대한 조언을 구하는 내용입니다."


In [31]:
import pandas as pd
from IPython.display import display
from tqdm.notebook import tqdm

# --- 1. 데이터 및 퓨샷 예제 준비 ---
print("✅ 테스트용 샘플 데이터를 다시 준비합니다...")
sample_data = {
    'english_dialogue': [
        "summarize: topic: Evening Invitation dialogue: #Person1#: Hello, is Jack there? #Person2#: This is him. #Person1#: Jack! It's Rose. #Person2#: Hey, Rose. How have you been? #Person1#: I've been great, thanks for asking. I'm hosting a few friends for dinner this Saturday. I was wondering if you could join us. #Person2#: That sounds great. What time should I arrive? #Person1#: Six o'clock works for me.",
        "summarize: topic: Job Interview Preparation dialogue: #Person1#: Hey, what should I wear to a job interview? #Person2#: I'd recommend going with a suit and tie. #Person1#: I'm worried I might get too nervous during the interview. #Person2#: Don't worry about it. Just do your best and showcase yourself confidently.",
    ]
}
test_df = pd.DataFrame(sample_data)

# 퓨샷 프롬프트에 사용할 '모범 답안' 예시
sample_dialogue_for_fewshot = test_df.iloc[0]['english_dialogue']
# Solar API가 한국어로 요약하도록 할 것이므로, 예시 요약문도 한국어여야 함
sample_summary_for_fewshot_ko = "로즈는 잭을 저녁 식사에 초대했고, 잭은 그 초대에 동의했습니다."
print("✅ 샘플 데이터 준비 완료.")

# --- 2. 퓨샷 프롬프트 2종 정의 ---

# 퓨샷 1 (내가 제안했던 버전)
FEW_SHOT_PROMPT_1 = f"""
You are a expert in the field of dialogue summarization. Following the sample's style, provide a concise Korean summary of the given dialogue.

Sample Dialogue:
{sample_dialogue_for_fewshot}

Sample Summary (Korean):
{sample_summary_for_fewshot_ko}
---
Dialogue to summarize:
{{dialogue_to_summarize}}

Korean Summary:
"""

# 퓨샷 2 (대회 베이스라인 스타일 버전)
FEW_SHOT_PROMPT_2 = f"""
You are a expert in the field of dialogue summarization, summarize the given dialogue in a concise manner. Follow the user's instruction carefully and provide a summary that is relevant to the dialogue.

Following the instructions below, summarize the given document.
Instructions:
1. Read the provided sample dialogue and corresponding summary.
2. Read the dialogue to summarize carefully.
3. Following the sample's style of summary, provide a concise summary of the given dialogue, in Korean.

Sample Dialogue:
{sample_dialogue_for_fewshot}

Sample Summary:
{sample_summary_for_fewshot_ko}
---
Dialogue:
{{dialogue_to_summarize}}

Summary:
"""

prompt_templates = {
    "퓨샷 프롬프트 1 (내 버전)": FEW_SHOT_PROMPT_1,
    "퓨샷 프롬프트 2 (베이스라인 버전)": FEW_SHOT_PROMPT_2,
}
print("✅ 퓨샷 프롬프트 2종 준비 완료.")

# --- 3. 퓨샷 A/B 테스트 실행 ---
print("\n🚀 퓨샷 프롬프트 최종 대결을 시작합니다...")
few_shot_results = []
for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="퓨샷 테스트 중"):
    english_dialogue = row['english_dialogue']
    result_row = {'Original English Dialogue': english_dialogue}
    
    for name, template in prompt_templates.items():
        korean_summary = get_translation(english_dialogue, template) # get_translation 함수는 4번 셀에 정의되어 있음
        result_row[name] = korean_summary
        
    few_shot_results.append(result_row)

print("\n✅ 퓨샷 대결 완료!")

# --- 4. 결과 출력 ---
few_shot_results_df = pd.DataFrame(few_shot_results)
pd.set_option('display.max_colwidth', None)

print("\n--- 퓨샷 프롬프트 비교 결과 ---")
display(few_shot_results_df)

✅ 테스트용 샘플 데이터를 다시 준비합니다...
✅ 샘플 데이터 준비 완료.
✅ 퓨샷 프롬프트 2종 준비 완료.

🚀 퓨샷 프롬프트 최종 대결을 시작합니다...


퓨샷 테스트 중:   0%|          | 0/2 [00:00<?, ?it/s]


✅ 퓨샷 대결 완료!

--- 퓨샷 프롬프트 비교 결과 ---


,Original English Dialogue,퓨샷 프롬프트 1 (내 버전),퓨샷 프롬프트 2 (베이스라인 버전)
0,"summarize: topic: Evening Invitation dialogue: #Person1#: Hello, is Jack there? #Person2#: This is him. #Person1#: Jack! It's Rose. #Person2#: Hey, Rose. How have you been? #Person1#: I've been great, thanks for asking. I'm hosting a few friends for dinner this Saturday. I was wondering if you could join us. #Person2#: That sounds great. What time should I arrive? #Person1#: Six o'clock works for me.","로즈는 잭을 토요일 저녁 식사에 초대했고, 잭은 6시에 참석하기로 했습니다.","로즈는 잭을 토요일 저녁 식사에 초대했고, 잭은 6시에 참석하기로 했습니다."
1,"summarize: topic: Job Interview Preparation dialogue: #Person1#: Hey, what should I wear to a job interview? #Person2#: I'd recommend going with a suit and tie. #Person1#: I'm worried I might get too nervous during the interview. #Person2#: Don't worry about it. Just do your best and showcase yourself confidently.","대화 상대는 취업 면접에 대해 이야기하며, 면접 복장과 면접 중 긴장하는 문제에 대해 조언을 나누었습니다.","직장인은 면접에 정장과 넥타이를 착용하기로 결정했고, 상대방은 그의 긴장을 이해하며 최선을 다하고 자신감 있게 자신을 표현하라고 격려했습니다."
